In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.set_option('display.max_columns', None)

#Data = pd.read_csv("../Data/smalldata.csv")
Data = pd.read_csv("~/data/project/300k.csv")
#print(Data.head())

#Steage 1
Selection = ['pokemonId', 'latitude', 'longitude','appearedTimeOfDay', 'appearedHour','appearedMinute',\
             'appearedDay','terrainType', 'closeToWater', 'gymDistanceKm','city', 'continent','weather', 'temperature', \
             'windSpeed', 'windBearing', 'pressure','population_density','urban','suburban','midurban','pokestopDistanceKm','rural']
#Selection = ['pokemonId', 'city','latitude', 'longitude', 'appearedHour','appearedMinute',\
#            'terrainType', 'closeToWater', 'gymDistanceKm','weather', 'temperature', \
#             'windSpeed', 'windBearing', 'pressure',\
#            'class' ]
ReducedData = Data.loc[:, Selection]
ReducedData.head()

# Data Cleaning

In [ ]:
def Convertnumber(Data, field):
    Data[field]= Data[field].astype('category')
    catdict = dict(enumerate(Data[field].cat.categories))
    Data[field] = Data[field].cat.codes
    return catdict, Data

In [ ]:
weatherdict, ReducedData = Convertnumber(ReducedData, 'weather')
waterdict, ReducedData = Convertnumber(ReducedData, 'closeToWater')
appearedTimeOfDaydict, ReducedData = Convertnumber(ReducedData, 'appearedTimeOfDay')
citydict, ReducedData = Convertnumber(ReducedData, 'city')
continentdict, ReducedData = Convertnumber(ReducedData, 'continent')
urbandict, ReducedData = Convertnumber(ReducedData, 'urban')
suburbandict, ReducedData = Convertnumber(ReducedData, 'suburban')
midurbandict, ReducedData = Convertnumber(ReducedData, 'midurban')
ruraldict, ReducedData = Convertnumber(ReducedData, 'rural')
ReducedData.head()

In [ ]:
ReducedData['pokestopDistanceKm'] = ReducedData['pokestopDistanceKm'].convert_objects(convert_numeric=True).dropna()

# Features Correclation Analysis

In [ ]:
plt.subplots(figsize = (30,30))
corr = abs(ReducedData.corr())
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            annot=True,cmap = "Blues")
plt.savefig('corr_figure.jpg')

plt.show()

# Behaviour Similarity Analysis 

In [ ]:
mean_col = ReducedData.groupby('pokemonId').mean()

In [ ]:
mean_col.head()

In [ ]:
DataForBehavior = mean_col
DataForBehavior.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances

In [ ]:
plt.subplots(figsize = (300,300))
#similaritymatrix = cosine_similarity(DataForBehavior)
similaritymatrix =  manhattan_distances(DataForBehavior)
df = pd.DataFrame(similaritymatrix)
sns.heatmap(df, 
            xticklabels=df.columns.values,
            yticklabels=df.columns.values,
            annot=True)
plt.savefig('BehaviorSimilar.jpg')
#plt.show()

In [ ]:
df.head()

# Find k Most Likely Pokemons

In [ ]:
import numpy as np

In [ ]:
 k = 10

In [ ]:
a = similaritymatrix
klike_matrix = np.argpartition(a,k+1)[:,:k+1]
klike = pd.DataFrame(np.argpartition(a,k+1)[:,:k+1])
klike.head()

In [ ]:
edges = []
for i in range(np.shape(a)[0]):
    for j in range(k+1):
        if i != klike_matrix[i,j]:
            pair = [i,klike_matrix[i,j]]
            edges.append(pair)

In [ ]:
vertices = klike.T.columns.values

In [ ]:
np.savetxt("edges.csv", edges, delimiter=",")
np.savetxt("vertices.csv", vertices, delimiter=",")

In [ ]:
Data = pd.read_csv("~/data/project/300k.csv")